In [ ]:
!nvidia-smi

Mon Jul  6 11:05:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash
pip3 --quiet install  torch torchvision pillow gitpython lz4 matplotlib numpy pyyaml scikit-learn six tqdm h5py opencv-python googledrivedownloader
pip3 --quiet install affine geojson shapely pyproj
pip uninstall thelper
rm -rf ./thelper
git clone https://github.com/sfoucher/thelper
cd thelper
pip install -v -e .


Created temporary directory: /tmp/pip-ephem-wheel-cache-j0svx12j
Created temporary directory: /tmp/pip-req-tracker-psskif_5
Created requirements tracker '/tmp/pip-req-tracker-psskif_5'
Created temporary directory: /tmp/pip-install-h8xrgdzf
Obtaining file:///content/thelper
  Added file:///content/thelper to build tracker '/tmp/pip-req-tracker-psskif_5'
    Running setup.py (path:/content/thelper/setup.py) egg_info for package from file:///content/thelper
  Source in /content/thelper has version 0.5.0rc0, which satisfies requirement thelper==0.5.0rc0 from file:///content/thelper
  Removed thelper==0.5.0rc0 from file:///content/thelper from build tracker '/tmp/pip-req-tracker-psskif_5'
  Running setup.py develop for thelper
Cleaning up...
Removed build tracker '/tmp/pip-req-tracker-psskif_5'


Cloning into 'thelper'...
    Running command python setup.py egg_info
    running egg_info
    creating thelper.egg-info
    writing thelper.egg-info/PKG-INFO
    writing dependency_links to thelper.egg-info/dependency_links.txt
    writing entry points to thelper.egg-info/entry_points.txt
    writing requirements to thelper.egg-info/requires.txt
    writing top-level names to thelper.egg-info/top_level.txt
    writing manifest file 'thelper.egg-info/SOURCES.txt'
    reading manifest template 'MANIFEST.in'
    /usr/local/lib/python3.6/dist-packages/setuptools/dist.py:454: UserWarning: Normalizing '0.5.0-rc' to '0.5.0rc0'
      warnings.warn(tmpl.format(**locals()))
    writing manifest file 'thelper.egg-info/SOURCES.txt'
    Running command /usr/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/content/thelper/setup.py'"'"'; __file__='"'"'/content/thelper/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n

In [ ]:
!pip install pyyaml yamlmagic
%load_ext yamlmagic

In [ ]:
%%bash
pip --quiet install pretrainedmodels albumentations opencv-python opencv-contrib-python pyyaml
git clone https://github.com/sfoucher/pytorch-segmentation


Cloning into 'pytorch-segmentation'...


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
%matplotlib inline
import os
import numpy as np
import cv2 as cv
import torch
import torchvision
import zipfile
import tarfile
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics
import time
import collections
import tqdm
import json
import requests
import zipfile
import gdal
import thelper
import sys
sys.path.append('/content/pytorch-segmentation/src')

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import os.path as osp
import sys


def maybe_download_and_extract(file_id, dest_path):
    filename = dest_path.split('/')[-1]
    file_path = dest_path
    download_dir= osp.dirname(osp.abspath(dest_path))
    if not os.path.isfile(dest_path):
      gdd.download_file_from_google_drive(file_id= file_id, dest_path= file_path)
      print("Download finished. Extracting files.")

      if file_path.endswith(".zip"):
          # Unpack the zip-file.
          zipfile.ZipFile(file=file_path, mode="r").extractall(download_dir)
      elif file_path.endswith((".tar.gz", ".tgz")):
          # Unpack the tar-ball.
          tarfile.open(name=file_path, mode="r:gz").extractall(download_dir)
      print("Done.")
    else:
        print("Data has apparently already been downloaded and unpacked.")
maybe_download_and_extract(file_id="1zyqOXH5G2I-iZCYIkt1EMDrDgnXKUD17", dest_path="/content/pytorch-segmentation/data/dataset.tar.gz")

Download finished. Extracting files.
Done.


In [ ]:
%%bash
cd /content/pytorch-segmentation/tf_model
wget http://download.tensorflow.org/models/deeplabv3_cityscapes_train_2018_02_06.tar.gz
tar -xvf deeplabv3_cityscapes_train_2018_02_06.tar.gz
cd ../src
python -m converter.convert_xception65 ../tf_model/deeplabv3_cityscapes_train/model.ckpt 19 ../model/cityscapes_deeplab_v3_plus/model.pth

deeplabv3_cityscapes_train/
deeplabv3_cityscapes_train/model.ckpt.index
deeplabv3_cityscapes_train/frozen_inference_graph.pb
deeplabv3_cityscapes_train/model.ckpt.data-00000-of-00001


--2020-07-05 16:14:18--  http://download.tensorflow.org/models/deeplabv3_cityscapes_train_2018_02_06.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.215.128, 2607:f8b0:400c:c0c::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.215.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460310119 (439M) [application/x-tar]
Saving to: ‘deeplabv3_cityscapes_train_2018_02_06.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  132M 3s
    50K .......... .......... .......... .......... ..........  0%  124M 3s
   100K .......... .......... .......... .......... ..........  0%  255M 3s
   150K .......... .......... .......... .......... ..........  0%  161M 3s
   200K .......... .......... .......... .......... ..........  0%  190M 3s
   250K .......... .......... .......... .......... ..........  0%  192M 3s
   300K .......... .......... .......... .......... ..........  0%  189M 3s
   350K 

In [ ]:
# load the magic 
%load_ext yamlmagic

In [ ]:
import sys
sys.path.append('/content/pytorch-segmentation/src')

In [ ]:
!cd /content/pytorch-segmentation/src
from models.net import EncoderDecoderNet, SPPNet
from losses.multi import MultiClassCriterion
from logger.log import debug_logger
from logger.plot import history_ploter
from utils.optimizer import create_optimizer
from utils.metrics import compute_iou_batch
import yaml
import pickle
import argparse
import yaml
import numpy as np
import albumentations as albu
from collections import OrderedDict
from pathlib import Path
from tqdm import tqdm

In [ ]:
%%yaml config
Net:
  enc_type: 'xception65'
  dec_type: 'unet'
  output_stride: 8
  output_channels: 19
Data:
  dataset: 'deepglobe-dynamic'
  target_size: (512, 512)
Train:
  max_epoch: 500
  batch_size: 4
  fp16: True
  resume: False
  pretrained_path: '/content/pytorch-segmentation/model/cityscapes_deeplab_v3_plus/model.pth'
  freeze: True
  seed: False
Loss:
  loss_type: 'Lovasz'
Optimizer:
  mode: 'sgd'
  base_lr: 0.007
  t_max: 30

<IPython.core.display.Javascript object>

### Prepare Training

In [ ]:
#config_path = '/content/pytorch-segmentation/config/deepglobe_deeplabv3_weights-cityscapes_19-outputs_small-patches_dynamic.yaml'

#with open(config_path) as yaml_file:
#  config = yaml.load(yaml_file)
net_config = config['Net']
data_config = config['Data']
train_config = config['Train']
loss_config = config['Loss']
opt_config = config['Optimizer']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
t_max = opt_config['t_max']

# Collect training parameters
max_epoch = train_config['max_epoch']
batch_size = train_config['batch_size']
fp16 = train_config['fp16']
resume = train_config['resume']
pretrained_path = train_config['pretrained_path']
freeze_enabled = train_config['freeze']
seed_enabled = train_config['seed']
dataset_base_dir = '/content/pytorch-segmentation/data/deepglobe_as_pascalvoc/VOCdevkit/VOC2012'
#########################################
# Deterministic training
if seed_enabled:
    seed = 100
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed=seed)
    import random
    random.seed(a=100)
#########################################

# Network
if 'unet' in net_config['dec_type']:
    net_type = 'unet'
    net_config.pop('output_stride')
    net_config['dec_type']= 'unet_scse'
    net_config['enc_type']= 'resnet18'
    model = EncoderDecoderNet(**net_config)
else:
    net_type = 'deeplab'
    model = SPPNet(**net_config)

dataset = data_config['dataset']
print(dataset)
if dataset == 'pascal':
    from dataset.pascal_voc import PascalVocDataset as Dataset
    net_config['output_channels'] = 21
    classes = np.arange(1, 22)
elif dataset == 'cityscapes':
    from dataset.cityscapes import CityscapesDataset as Dataset
    net_config['output_channels'] = 19
    classes = np.arange(1, 20)
elif dataset == 'deepglobe-dynamic':
    from dataset.deepglobe_dynamic import DeepGlobeDatasetDynamic as Dataset
    net_config['output_channels'] = 7
    classes = np.arange(1, 8)
else:
    raise NotImplementedError
del data_config['dataset']

modelname = 'test' #config_path.stem
output_dir = Path('/content/pytorch-segmentation/model') / modelname
output_dir.mkdir(exist_ok=True)
log_dir = Path('/content/pytorch-segmentation/logs') / modelname
log_dir.mkdir(exist_ok=True)

logger = debug_logger(log_dir)
logger.debug(config)
logger.info(f'Device: {device}')
logger.info(f'Max Epoch: {max_epoch}')

# Loss
loss_fn = MultiClassCriterion(**loss_config).to(device)
params = model.parameters()
optimizer, scheduler = create_optimizer(params, **opt_config)

# history
if resume:
    with open(log_dir.joinpath('history.pkl'), 'rb') as f:
        history_dict = pickle.load(f)
        best_metrics = history_dict['best_metrics']
        loss_history = history_dict['loss']
        iou_history = history_dict['iou']
        start_epoch = len(iou_history)
        for _ in range(start_epoch):
            scheduler.step()
else:
    start_epoch = 0
    best_metrics = 0
    loss_history = []
    iou_history = []

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


2020-07-06 11:18:21,376 train 70 [INFO][<module>] Device: cuda:0
2020-07-06 11:18:21,377 train 71 [INFO][<module>] Max Epoch: 500


deepglobe-dynamic


In [ ]:
# Dataset
affine_augmenter = albu.Compose([albu.HorizontalFlip(p=.5),
                                 # Rotate(5, p=.5)
                                 ])
# image_augmenter = albu.Compose([albu.GaussNoise(p=.5),
#                                 albu.RandomBrightnessContrast(p=.5)])
image_augmenter = None
"""
train_dataset = Dataset(base_dir= '/content/pytorch-segmentation/data/deepglobe_as_pascalvoc/VOCdevkit/VOC2012',
                        affine_augmenter=affine_augmenter, image_augmenter=image_augmenter,
                        net_type=net_type, **data_config)
valid_dataset = Dataset(base_dir= '/content/pytorch-segmentation/data/deepglobe_as_pascalvoc/VOCdevkit/VOC2012',
                        split='valid', net_type=net_type, **data_config)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4,
                          pin_memory=True, drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)
"""

"\ntrain_dataset = Dataset(base_dir= '/content/pytorch-segmentation/data/deepglobe_as_pascalvoc/VOCdevkit/VOC2012',\n                        affine_augmenter=affine_augmenter, image_augmenter=image_augmenter,\n                        net_type=net_type, **data_config)\nvalid_dataset = Dataset(base_dir= '/content/pytorch-segmentation/data/deepglobe_as_pascalvoc/VOCdevkit/VOC2012',\n                        split='valid', net_type=net_type, **data_config)\ntrain_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4,\n                          pin_memory=True, drop_last=True)\nvalid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)\n"

In [ ]:
# To device
model = model.to(device)
# Pretrained model
if 'unet' not in net_config['dec_type'] and pretrained_path:
    logger.info(f'Resume from {pretrained_path}')
    param = torch.load(pretrained_path)
    model.load_state_dict(param)
    del param

    #########################################
    if freeze_enabled:
        # Code de Rémi
        # Freeze layers
        for param_index in range(int((len(optimizer.param_groups[0]['params'])) * 0.5)):
            optimizer.param_groups[0]['params'][param_index].requires_grad = False
    #########################################


In [ ]:
# fp16
if fp16:
    # I only took the necessary files because I don't need the C backend of apex,
    # which is broken and can't be installed
    # from apex import fp16_utils
    from utils.apex.apex.fp16_utils.fp16util import BN_convert_float
    from utils.apex.apex.fp16_utils.fp16_optimizer import FP16_Optimizer
    # model = fp16_utils.BN_convert_float(model.half())
    model = BN_convert_float(model.half())
    # optimizer = fp16_utils.FP16_Optimizer(optimizer, verbose=False, dynamic_loss_scale=True)
    optimizer = FP16_Optimizer(optimizer, verbose=False, dynamic_loss_scale=True)
    logger.info('Apply fp16')


2020-07-06 11:21:12,710 train 12 [INFO][<module>] Apply fp16


In [ ]:
# Restore model
if resume:
    model_path = output_dir.joinpath(f'model_tmp.pth')
    logger.info(f'Resume from {model_path}')
    param = torch.load(model_path)
    model.load_state_dict(param)
    del param
    opt_path = output_dir.joinpath(f'opt_tmp.pth')
    param = torch.load(opt_path)
    optimizer.load_state_dict(param)
    del param

In [ ]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [ ]:

# Train
for i_epoch in range(start_epoch, max_epoch):
    logger.info(f'Epoch: {i_epoch}')
    logger.info(f'Learning rate: {optimizer.param_groups[0]["lr"]}')

    train_losses = []
    train_ious = []
    model.train()

    # Initialize randomized but balanced datasets
    train_dataset = Dataset(base_dir = dataset_base_dir,
                            affine_augmenter=affine_augmenter, image_augmenter=image_augmenter,
                            net_type=net_type, **data_config)
    valid_dataset = Dataset(base_dir = dataset_base_dir,
                            split='valid', net_type=net_type, **data_config)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4,
                              pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

    with tqdm(train_loader) as _tqdm:
        for batched in _tqdm:
            images, labels = batched
            if fp16:
                images = images.half()
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            preds = model(images)
            if net_type == 'deeplab':
                preds = F.interpolate(preds, size=labels.shape[1:], mode='bilinear', align_corners=True)
            if fp16:
                loss = loss_fn(preds.float(), labels)
            else:
                loss = loss_fn(preds, labels)

            preds_np = preds.detach().cpu().numpy()
            labels_np = labels.detach().cpu().numpy()
            iou = compute_iou_batch(np.argmax(preds_np, axis=1), labels_np, classes)

            _tqdm.set_postfix(OrderedDict(seg_loss=f'{loss.item():.5f}', iou=f'{iou:.3f}'))
            train_losses.append(loss.item())
            train_ious.append(iou)

            if fp16:
                optimizer.backward(loss)
            else:
                loss.backward()
            optimizer.step()

    scheduler.step()

    train_loss = np.mean(train_losses)
    train_iou = np.nanmean(train_ious)
    logger.info(f'train loss: {train_loss}')
    logger.info(f'train iou: {train_iou}')

    torch.save(model.state_dict(), output_dir.joinpath('model_tmp.pth'))
    torch.save(optimizer.state_dict(), output_dir.joinpath('opt_tmp.pth'))

    valid_losses = []
    valid_ious = []
    model.eval()
    with torch.no_grad():
        with tqdm(valid_loader) as _tqdm:
            for batched in _tqdm:
                images, labels = batched
                if fp16:
                    images = images.half()
                images, labels = images.to(device), labels.to(device)
                preds = model.tta(images, net_type=net_type)
                if fp16:
                    loss = loss_fn(preds.float(), labels)
                else:
                    loss = loss_fn(preds, labels)

                preds_np = preds.detach().cpu().numpy()
                labels_np = labels.detach().cpu().numpy()

                # I changed a parameter in the compute_iou method to prevent it from yielding nans
                iou = compute_iou_batch(np.argmax(preds_np, axis=1), labels_np, classes)

                _tqdm.set_postfix(OrderedDict(seg_loss=f'{loss.item():.5f}', iou=f'{iou:.3f}'))
                valid_losses.append(loss.item())
                valid_ious.append(iou)

    valid_loss = np.mean(valid_losses)
    valid_iou = np.mean(valid_ious)
    logger.info(f'valid seg loss: {valid_loss}')
    logger.info(f'valid iou: {valid_iou}')

    if best_metrics < valid_iou:
        best_metrics = valid_iou
        logger.info('Best Model!')
        torch.save(model.state_dict(), output_dir.joinpath('model.pth'))
        torch.save(optimizer.state_dict(), output_dir.joinpath('opt.pth'))

    loss_history.append([train_loss, valid_loss])
    iou_history.append([train_iou, valid_iou])
    history_ploter(loss_history, log_dir.joinpath('loss.png'))
    history_ploter(iou_history, log_dir.joinpath('iou.png'))

    history_dict = {'loss': loss_history,
                    'iou': iou_history,
                    'best_metrics': best_metrics}
    with open(log_dir.joinpath('history.pkl'), 'wb') as f:
        pickle.dump(history_dict, f)


2020-07-06 11:21:40,724 train 4 [INFO][<module>] Epoch: 0
2020-07-06 11:21:40,725 train 5 [INFO][<module>] Learning rate: 0.007
  0%|          | 0/417 [00:00<?, ?it/s]/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add is deprecated:
	add(Tensor input, Number alpha, Tensor other, *, Tensor out)
Consider using one of the following signatures instead:
	add(Tensor input, Tensor other, *, Number alpha, Tensor out)
  4%|▍         | 17/417 [20:24<8:00:19, 72.05s/it, seg_loss=0.94565, iou=0.009]


KeyboardInterrupt: ignored

In [ ]:
with tqdm(train_loader) as _tqdm:
  for batched in _tqdm:
      images, labels = batched
      if fp16:
          images = images.half()
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()
      preds = model(images)
      if net_type == 'deeplab':
          preds = F.interpolate(preds, size=labels.shape[1:], mode='bilinear', align_corners=True)
      if fp16:
          loss = loss_fn(preds.float(), labels)
      else:
          loss = loss_fn(preds, labels)

      preds_np = preds.detach().cpu().numpy()
      labels_np = labels.detach().cpu().numpy()
      iou = compute_iou_batch(np.argmax(preds_np, axis=1), labels_np, classes)

      _tqdm.set_postfix(OrderedDict(seg_loss=f'{loss.item():.5f}', iou=f'{iou:.3f}'))
      train_losses.append(loss.item())
      train_ious.append(iou)

      if fp16:
          optimizer.backward(loss)
      else:
          loss.backward()
      optimizer.step()

  0%|          | 0/834 [00:05<?, ?it/s]


RuntimeError: ignored

In [ ]:
print(pretrained_path)

../model/cityscapes_deeplab_v3_plus/model.pth
